In [5]:
%matplotlib inline
import os
import numpy as np
import platform
from matplotlib import pyplot as plt
from cartopy import crs as ccrs
from scipy import linalg
from skimage.transform import resize
from mintpy.utils import ptime, readfile, writefile, plot as pp, utils as ut
from mintpy.asc_desc2horz_vert import get_design_matrix4east_north_up
from mintpy.ifgram_inversion import skip_invalid_obs
from mintpy.cli import view, image_stitch, save_gdal
from mintpy.view import prep_slice, plot_slice
plt.rcParams.update({'font.size': 12})

# platform
if platform.system() == 'Linux':
    proj_dir = os.path.expanduser('~/data/2024NotoEQ')
else:
    proj_dir = os.path.expanduser('~/data/archives/2024NotoEQ')

work_dir = os.path.join(proj_dir, 'data')
os.chdir(work_dir)
print('Go to directory:', work_dir)

# output grid
S, N, W, E = 36.7, 37.6, 136.6, 137.45
ref_lat, ref_lon = 36.83, 137.01

# output
dis_enu_file = os.path.join(work_dir, 'defo3d/dis_enu.h5')
dis_enu_std_file = os.path.join(work_dir, 'defo3d/dis_enu_std.h5')

Go to directory: /home/yunjunz/data/2024NotoEQ/data


### 1. Read displacement and geometry

In [6]:
# inputs
# ALL files MUST be geocoded into the same bounding box (SNWE)
# The 1st file is used as the grid size reference for the output
dis_files = [
    os.path.join(work_dir, 'ALOS2_A121_20220926_20240101/dense_offset/geo/offRg.geo'),
    os.path.join(work_dir, 'ALOS2_D026_20230606_20240102/dense_offset/geo/offRg.geo'),
    os.path.join(work_dir, 'ALOS2_A127_20231206_20240103/dense_offset/geo/offRg.geo'),
    os.path.join(work_dir, 'ALOS2_A128_20230612_20240108/dense_offset/geo/offRg.geo'),
    os.path.join(work_dir, 'ALOS2_D019_20211019_20240109/dense_offset/geo/offRg.geo'),
]
dis_std_files = [
    os.path.join(work_dir, 'ALOS2_A121_20220926_20240101/dense_offset/geo/offRgStd.geo'),
    os.path.join(work_dir, 'ALOS2_D026_20230606_20240102/dense_offset/geo/offRgStd.geo'),
    os.path.join(work_dir, 'ALOS2_A127_20231206_20240103/dense_offset/geo/offRgStd.geo'),
    os.path.join(work_dir, 'ALOS2_A128_20230612_20240108/dense_offset/geo/offRgStd.geo'),
    os.path.join(work_dir, 'ALOS2_D019_20211019_20240109/dense_offset/geo/offRgStd.geo'),
]
enu_files = [os.path.join(os.path.dirname(x), 'enu.geo') for x in dis_files]
num_file = len(dis_files)

### 2. Estimate 3D displacement field

In [ ]:
def estimate_enu_displacement(dis_los, dis_los_std, G, weight=False):

    dis_los = dis_los.reshape(G.shape[0], -1)
    dis_los_std = dis_los_std.reshape(G.shape[0], -1)

    # initial output value
    dis_enu     = np.full(3, np.nan, dtype=np.float32)
    dis_enu_std = np.full(3, np.nan, dtype=np.float32)

    # skip pixels with none in all files
    if np.all(np.isnan(dis_los)):
        return dis_enu, dis_enu_std

    # skip invalid observations in some files
    dis_los, [G, dis_los_std] = skip_invalid_obs(dis_los, mat_list=[G, dis_los_std])

    # invert
    if weight:
        w_sqrt = 1. / dis_los_std
        X, e2 = linalg.lstsq(np.multiply(G, w_sqrt), np.multiply(dis_los, w_sqrt), cond=1e-5)[:2]
    else:
        X, e2 = linalg.lstsq(G, dis_los, cond=1e-5)[:2]

    if e2.size != 0:
        dis_enu = X.flatten()

        # calculate inversion quality
        Gplus = linalg.pinv(G)
        dis_los_cov = np.diag(np.square(dis_los_std.flatten()))
        dis_enu_cov = np.linalg.multi_dot([Gplus, dis_los_cov, Gplus.T])
        dis_enu_std = np.sqrt(np.diag(dis_enu_cov))

    return dis_enu, dis_enu_std

### 3. Plot